## 3- MLflow Model Registry

## 3.1. Model Registry

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
model_id = "m-d8638308e151497b8b049622c8a3cf6a" # Find this model_id in your mlflow ui first and replace it
model_name = "CatClassifier" # Choose a name for your model
model_uri = f"models:/{model_id}"
mlflow.register_model(model_uri=model_uri, name=model_name)

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
model_id = "m-abc9b2927a53483f8f1ba1c2c37b187a" # Find this model_id in your mlflow ui first and replace it
model_name = "CatDogClassifier" # Choose a name for your model
model_uri = f"models:/{model_id}"
mlflow.register_model(model_uri=model_uri, name=model_name)

## 3.2. Version Search

In [ ]:
from mlflow.tracking import MlflowClient
model_name = "CatDogClassifier"
client = MlflowClient(tracking_uri="http://127.0.0.1:5000/")

latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

## 3.3. Stage Transition

In [ ]:
model_version=1
new_stage = "Staging"
model_name = "CatDogClassifier"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

In [ ]:
model_version=1
new_stage = "Production"
model_name = "CatDogClassifier"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=True
)

## 3.4. Update MetaData for a Model

In [ ]:
from datetime import datetime
model_version=1
model_name = "CatDogClassifier"
date = datetime.today().date()
new_description = f"The model version {model_version} was transitioned to production on {date}"

client.update_model_version(
    name=model_name,
    version=model_version,
    description=new_description
)

## 3.5. Load Model based on Stage and Model Name

In [23]:
model_name = "CatDogClassifier"
stage = "Production"  # Can be None, "Staging", "Production", or "Archived"

model = mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")


## 3.6. Model Load and Execute

In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import kagglehub
import mlflow

def load_data(kagglehub_path: str):
    path = kagglehub.dataset_download(kagglehub_path)
    df = pd.read_csv(path + "/housing.csv")
    df.dropna(inplace=True)
    xtrain, xtest, ytrain, ytest = train_test_split(df.drop(columns=['median_house_value','ocean_proximity']), 
                                                    df['median_house_value'], test_size=0.4, random_state=42)
    return xtrain, xtest, ytrain, ytest

%time print(f'rmse: {mean_squared_error(ytest, model.predict(xtest))}')

rmse: 4660334167.735469
CPU times: user 1.92 ms, sys: 1.44 ms, total: 3.36 ms
Wall time: 2.77 ms
